##  <span style='color:green '>NFL Fantasy Football Draft Position Predictor</span>
<span style='color:green '>This script runs the models from NFL_Fantasy_model_Tuner_woQB and NFL_Fantasy_model_Tuner_QB  
    This notebook exists for debugging purposes only see NFL_Fantasy_Predictor.py for deployment. This notebook became NFL_Fantasy_Predictor.py.  
    laf 08.24.2022
</span>

In [43]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import joblib
import sys

###  <span style='color:green '> Load NFL Fantasy Prediction Model</span>

In [44]:
draft = joblib.load("Resources/draft_position_no_QB.joblib")
qbdraft = joblib.load("Resources/draft_position_qb.joblib")

###  <span style='color:green '>Load Data for Predictive Models</span>
####  <span style='color:green '>Skill Players Not Including Quarterbacks </span>

In [45]:
# This file is the output from NFL Data Cleaned.py 
adp = pd.read_csv('Resources/ADPxFinal.csv') 

# Drop players ineligible to be drafted 
adp.dropna(subset=['AVG'], inplace=True)

# Preserve label information for Output file 
adp_scope = adp[['Player',
                 '2019 FantasyPoints',
                 '2020 FantasyPoints',
                 '2021 FantasyPoints',
                 'Production21',
                 'Average Total Production',
                 '2021 Tm',
                 'Pos',
                 'AVG'
                ]].copy()

####  <span style='color:green '>Quarterbacks </span>

In [55]:
# This file is the output from NFL Data Cleaned.py 
qb = pd.read_csv('Resources/QBxADPxFinal.csv') 

# Drop players ineligible to be drafted 
qb.dropna(subset=['AVG'], inplace=True)

# Preserve label information for Output file 
qb_scope = qb[['Player',
               '2019 FantasyPoints',
               '2020 FantasyPoints',
               '2021 FantasyPoints',
               'Production21',
               'Average Total Production',
               '2021 Tm',
               'Pos',
               'AVG'
                ]].copy()

###  <span style='color:green '>Get Data Ready </span>

 <span style='color:green '>* Drop players ineligible to be drafted    
    * Look for new invalid data   
    * Remove all rows that contain values Infinity and -Infinity 
    * Remove unnamed column and player name  
    * Preserve label information for Output file  
    * Scale our new data </span>

In [67]:
# Drop unnamed column, Player, Pos and 2021 Team
col = [0,1,2,3]
adp.drop(adp.columns[col],axis=1,inplace=True)
qb.drop(qb.columns[col],axis=1,inplace=True)

In [48]:
# All other skill 
# Verify that all numeric data is numeric 
invalidNumbers = adp[~adp.applymap(np.isreal).all(1)]
if len(invalidNumbers) > 0:
    sys.exit(f'There are {len(invalidNumbers)} rows with invalid numeric data')
    
# Check for unexpected nulls 
count_nan = adp.isna().sum().sum()
if count_nan > 0:
    sys.exit(f'Invalid data Encountered: {count_nan} fields have null values')

In [49]:
# QB 
# Verify that all numeric data is numeric 
invalidNumbers = qb[~qb.applymap(np.isreal).all(1)]
if len(invalidNumbers) > 0:
    sys.exit(f'There are {len(invalidNumbers)} rows with invalid numeric data')
    
# Check for unexpected nulls 
count_nan = qb.isna().sum().sum()
if count_nan > 0:
    sys.exit(f'Invalid data Encountered: {count_nan} fields have null values')

In [50]:
# Drop rows that contain infinity values in our Results Dataset 
adp_scope.replace([np.inf, -np.inf], 'drop', inplace=True)
adp_scope = adp_scope[~adp_scope.eq('drop').any(1)]

qb_scope.replace([np.inf, -np.inf], 'drop', inplace=True)
qb_scope = qb_scope[~qb_scope.eq('drop').any(1)]

# Drop rows that contain infinity values in our Prediction Dataset 
adp.replace([np.inf, -np.inf], np.nan, inplace=True)
adp.dropna(inplace=True)

adp.replace([np.inf, -np.inf], np.nan, inplace=True)
adp.dropna(inplace=True)

In [51]:
# Standarize data with Scaler(s) required by models
qbs = StandardScaler().fit_transform(qb)
apds = MinMaxScaler().fit_transform(adp)

###  <span style='color:green '> Apply PCA </span> 

In [58]:
# Applying PCA to reduce dimensions to the number required by each model 
pca = PCA(n_components= 23)
pcaq = PCA(n_components= 13)

# Fit our new Principal Component Analysis reduced Features to our Model
pfa = pca.fit_transform(apds)
pfb = pcaq.fit_transform(qbs)

# Transform PCA data to a DataFrame
pf = pd.DataFrame(data=pfa)
pfqb = pd.DataFrame(data=pfb)

###  <span style='color:green '>Predict Draft Positions</span>

In [59]:
draft_position = draft.predict(pf)
qb_draft_position = qbdraft.predict(pfqb)

###  <span style='color:green '>Add predicted draft positions to our results file </span>

In [66]:
# Add column to results df
frames = [adp_scope, qb_scope]
adp_scope['Prediction'] = draft_position
qb_scope['Prediction'] = qb_draft_position

# Combine DataFrames into 1 complete dataset.
final = pd.concat(frames)

In [65]:
# Write file to csv 
final.to_csv('Resources/Draft.csv')